In [ ]:
import cv2
import nibabel as nib
import numpy as np
# from pycimg import CImg
# from scipy import ndimage as ndi
# from skimage import filters
# import surface_distance
import matplotlib.pyplot as plt
import os
import torch
from torch.optim import AdamW
from torch.nn import BCEWithLogitsLoss
from torch.utils.data import DataLoader
from torcheval.metrics import BinaryAUROC
from monai.networks.nets import DenseNet121
import torchvision.transforms as transforms

In [4]:
data_dir_3D = "Data/"
dateset_dir = "dataset_bodyslices"

In [5]:
class MedNISTDataset(torch.utils.data.Dataset):
    def __init__(self, data_dir_3D, data_dir_slices):
        self.data_dir_3D = data_dir_3D
        self.data_dir_slices = data_dir_slices
        self.total_length = 0
        self.labels = []
        images_3D_filenames = filter(lambda el: el.split('.')[-1] == 'gz', os.listdir(data_dir_3D))
        with open(data_dir_3D + "oznaczenia.txt") as f:
            lines = f.readlines()
            bounds = {}
            for line in lines:
                fname = line.split('\t')[0]
                left_bound = int(line.split('\t')[1])
                right_bound = int(line.split('\t')[2])
                bounds.update({fname: (left_bound, right_bound)})

        imwrite = False
        if not os.path.exists(data_dir_slices):
            imwrite = True
            os.mkdir(data_dir_slices)

        depths = {}
        for fname in images_3D_filenames:
            img = self._load_3D(fname)
            bounds_for_img = bounds.get(fname)
            labels_for_img = np.zeros(img.shape[-1])
            depths.update({fname: img.shape[-1]})
            labels_for_img[bounds_for_img[1]: bounds_for_img[0]] = 1
            for label in labels_for_img:
                self.labels.append(label)

            if imwrite:
                for i, slice in enumerate(img.T, self.total_length):
                    cv2.imwrite(os.path.join(data_dir_slices, self._get_fname_from_index(i)), slice)

            length = img.shape[-1]
            self.total_length += length

        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize(224),
            transforms.Normalize((0.5,), (0.5,))
        ])
        
        print(depths)
        

    def _get_fname_from_index(self, i):
        return str(i)+".png"

    def _load_3D(self, fname):
        img = nib.load(os.path.join(self.data_dir_3D, fname)).get_fdata()
        print("Loaded:", fname)
        return img
    
    def _load_slice(self, index):
        
        fname = self._get_fname_from_index(index)
        img = cv2.imread(os.path.join(self.data_dir_slices, fname), 0)
        return self.transform(img)

    def __len__(self):
        return self.total_length

    def __getitem__(self, index):
        return self._load_slice(index), self.labels[index]


full_ds = MedNISTDataset(data_dir_3D, dateset_dir)

Loaded: 0001_1_.nii.gz
Loaded: 0001_2_.nii.gz
Loaded: 0002_1_.nii.gz
Loaded: 0002_2_.nii.gz
Loaded: 0003_1_.nii.gz
Loaded: 0003_2_.nii.gz
Loaded: 0004_1_.nii.gz
Loaded: 0004_2_.nii.gz
Loaded: 0005_1_.nii.gz
Loaded: 0006_1_.nii.gz
Loaded: 0006_2_.nii.gz
Loaded: 0007_1_.nii.gz
Loaded: 0007_2_.nii.gz
Loaded: 0008_1_.nii.gz
Loaded: 0008_2_.nii.gz
Loaded: 0009_1_.nii.gz
Loaded: 0009_2_.nii.gz
Loaded: 0010_1_.nii.gz
Loaded: 0010_2_.nii.gz
{'0001_1_.nii.gz': 223, '0001_2_.nii.gz': 223, '0002_1_.nii.gz': 207, '0002_2_.nii.gz': 207, '0003_1_.nii.gz': 207, '0003_2_.nii.gz': 207, '0004_1_.nii.gz': 311, '0004_2_.nii.gz': 311, '0005_1_.nii.gz': 223, '0006_1_.nii.gz': 311, '0006_2_.nii.gz': 267, '0007_1_.nii.gz': 267, '0007_2_.nii.gz': 223, '0008_1_.nii.gz': 311, '0008_2_.nii.gz': 311, '0009_1_.nii.gz': 311, '0009_2_.nii.gz': 267, '0010_1_.nii.gz': 267, '0010_2_.nii.gz': 207}


In [6]:
train_ds, val_ds = torch.utils.data.random_split(full_ds, [0.8, 0.2])
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=0)
val_loader = DataLoader(val_ds, batch_size=32, shuffle=True, num_workers=0)

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

loss_fn = BCEWithLogitsLoss(pos_weight=torch.tensor(sum(full_ds.labels)/len(full_ds)))
model = DenseNet121(spatial_dims=2, in_channels=1, out_channels=2).to(device)
optimizer = AdamW(model.parameters(), lr=0.0001, weight_decay=0.01)

max_epochs = 5
auc_metric = BinaryAUROC()

In [8]:
epoch_loss_values = []
metric_values = []

for epoch in range(max_epochs):
    print(f"Epoch {epoch + 1}/{max_epochs}")
    model.train()
    epoch_loss = 0
    for i, batch_data in enumerate(train_loader):
        inputs, labels = batch_data[0].to(device), batch_data[1].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

        print(f"{i}/{len(full_ds) // train_loader.batch_size}, " f"train_loss: {loss.item():.4f}")
        epoch_len = len(full_ds) // train_loader.batch_size
    epoch_loss /= i
    epoch_loss_values.append(epoch_loss)
    print(f"epoch {epoch + 1} average loss: {epoch_loss:.4f}")

    model.eval()
    with torch.no_grad():
        for i, test_data in enumerate(val_loader):
            test_input, test_label = test_data[0].to(device), test_data[1].to(device)
            pred = model(test_input)
            auc_metric.update(pred, test_label)
            metric_values.append(auc_metric.compute())
        print(f"current epoch: {epoch + 1} current AUC: {auc_metric.compute():.4f}")

Epoch 1/5


ValueError: Target size (torch.Size([32])) must be the same as input size (torch.Size([32, 2]))

In [ ]:
plt.plot(epoch_loss_values)
plt.title("Loss by epoch")
plt.show()

NameError: name 'epoch_loss_values' is not defined

In [ ]:
######